In [6]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def extrapolate_sales_data(input_file, x, max_date, output_file):
    df = pd.read_csv(input_file)
    df['yearmonth'] = pd.to_datetime(df['yearmonth'], format='%Y%m')
    new_df = pd.DataFrame(columns=df.columns)
    for (customer, product), group in df.groupby(['customer', 'product']):
        group = group.sort_values('yearmonth')
        for i, row in group.iterrows():
            new_df = new_df.append(row)
            for j in range(1, x+1):
                yearmonth = row['yearmonth'] + pd.DateOffset(months=j)
                
                if yearmonth > pd.to_datetime(max_date, format='%Y%m'):
                    break
                if len(df[(df['customer']==customer) & (df['product']==product) & (df['yearmonth']==yearmonth)]) == 0:
                    new_row = {'customer': customer, 'product': product, 'yearmonth': yearmonth, 'sales': 0}
                    new_df = new_df.append(new_row, ignore_index=True)

    new_df['yearmonth'] = new_df['yearmonth'].dt.strftime('%Y%m').astype(int)

    new_df.drop_duplicates(inplace=True)
    new_df.sort_values(['customer', 'product', 'yearmonth'], inplace=True)
    new_df.to_csv(output_file, index=False)
    
extrapolate_sales_data('input.csv', 4, '201812', 'output.csv')
